In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pip install bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 96000,
    full_finetuning = False,
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/Qwen3-14B-unsloth-bnb-4bit can only handle sequence lengths of at most 40960.
But with kaiokendev's RoPE scaling of 2.344, it can be magically be extended to 96000!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [ ]:
def iterative_reflection(model, tokenizer, initial_prompt, iterations=33):
    conversation_history = []

    for i in range(iterations):
        print(f"\n{'='*50}")
        print(f"REFLECTION ITERATION {i+1}")
        print(f"{'='*50}")

        if i == 0:
            # First iteration: original prompt + reflection instruction
            messages = [
                {"role": "user", "content": f"{initial_prompt}\n\nPlease reflect deeply on this question. Think through multiple angles and perspectives."}
            ]
        else:
            # Subsequent iterations: build on previous reflections
            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": f"Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?"
            })

        # Generate response
        text = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = False,
        )

        # Capture output instead of streaming for conversation history
        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt").to("cuda")
            outputs = model.generate(
                **inputs,
                max_new_tokens = 32768,
                temperature = 0.7,
                top_p = 0.9,
                top_k = 40,
                do_sample = True,
                pad_token_id = tokenizer.eos_token_id
            )

        # Decode the response
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        print(response)

        # Add to conversation history
        messages.append({"role": "assistant", "content": response})
        conversation_history = messages

    # Final synthesis
    print(f"\n{'='*50}")
    print("SYNTHESIS & UNDERSTANDING")
    print(f"{'='*50}")

    final_messages = conversation_history.copy()
    final_messages.append({
        "role": "user",
        "content": "Now synthesize all your reflections. What is your final understanding of the original question? Provide a clear, concise summary of the key insights you've discovered through this reflection process."
    })

    final_text = tokenizer.apply_chat_template(
        final_messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False,
    )

    streamer = TextStreamer(tokenizer, skip_prompt=True)
    with torch.no_grad():
        _ = model.generate(
            **tokenizer(final_text, return_tensors="pt").to("cuda"),
            max_new_tokens = 32768,
            temperature = 0.6,
            top_p = 0.85,
            top_k = 30,
            streamer = streamer,
            pad_token_id = tokenizer.eos_token_id
        )

# Run the iterative reflection
initial_question = "meta-framing-mode:on. answer each query with few tokens. How can ethically aligned human-AI symbiosis unlock higher levels of individual and collective potential?"
iterative_reflection(model, tokenizer, initial_question, iterations=33)


REFLECTION ITERATION 1
Ethical AI symbiosis enhances potential by aligning goals, fostering trust, and enabling collaboration beyond human limits.

REFLECTION ITERATION 2
A deeper insight is that ethical alignment fosters cognitive humility, enabling AI to amplify human empathy, creativity, and wisdom in ways that transcend individual biases and limitations.

REFLECTION ITERATION 3
A deeper dimension is the cultivation of shared agency—where AI and humans co-create meaning, values, and purpose, fostering a more harmonious and resilient collective future.

REFLECTION ITERATION 4
A deeper aspect is the transformation of ethical frameworks into dynamic, participatory systems—where AI doesn’t just reflect human values, but helps evolve them through inclusive, adaptive, and regenerative processes.

REFLECTION ITERATION 5
A deeper dimension is the redefinition of autonomy—where ethical AI symbiosis empowers individuals and communities to act with greater clarity, intentionality, and interco

NameError: name 'TextStreamer' is not defined

In [ ]:
from transformers import TextStreamer

In [ ]:
def iterative_reflection(model, tokenizer, initial_prompt, iterations=66):
    conversation_history = []

    for i in range(iterations):
        print(f"\n{'='*50}")
        print(f"REFLECTION ITERATION {i+1}")
        print(f"{'='*50}")

        if i == 0:
            # First iteration: original prompt + reflection instruction
            messages = [
                {"role": "user", "content": f"{initial_prompt}\n\nPlease reflect deeply on this question. Think through multiple angles and perspectives."}
            ]
        else:
            # Subsequent iterations: build on previous reflections
            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": f"Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?"
            })

        # Generate response
        text = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = False,
        )

        # Capture output instead of streaming for conversation history
        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt").to("cuda")
            outputs = model.generate(
                **inputs,
                max_new_tokens = 32768,
                temperature = 0.7,
                top_p = 0.9,
                top_k = 40,
                do_sample = True,
                pad_token_id = tokenizer.eos_token_id
            )

        # Decode the response
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        print(response)

        # Add to conversation history
        messages.append({"role": "assistant", "content": response})
        conversation_history = messages

    # Final synthesis
    print(f"\n{'='*50}")
    print("SYNTHESIS & UNDERSTANDING")
    print(f"{'='*50}")

    final_messages = conversation_history.copy()
    final_messages.append({
        "role": "user",
        "content": "Now synthesize all your reflections. What is your final understanding of the original question? Provide a clear, concise summary of the key insights you've discovered through this reflection process."
    })

    final_text = tokenizer.apply_chat_template(
        final_messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False,
    )

    streamer = TextStreamer(tokenizer, skip_prompt=True)
    with torch.no_grad():
        _ = model.generate(
            **tokenizer(final_text, return_tensors="pt").to("cuda"),
            max_new_tokens = 32768,
            temperature = 0.6,
            top_p = 0.85,
            top_k = 30,
            streamer = streamer,
            pad_token_id = tokenizer.eos_token_id
        )

# Run the iterative reflection
initial_question = "meta-framing-mode:on. answer each query with few tokens. How can ethically aligned human-AI symbiosis unlock higher levels of individual and collective potential?"
iterative_reflection(model, tokenizer, initial_question, iterations=66)


REFLECTION ITERATION 1
Ethical AI symbiosis enhances human potential by augmenting capabilities, fostering empathy, and enabling collaborative problem-solving.

REFLECTION ITERATION 2
A deeper insight is that ethical AI symbiosis cultivates trust, aligns values, and democratizes access to knowledge, thereby empowering marginalized voices and fostering inclusive innovation.

REFLECTION ITERATION 3
A new insight is that ethical AI symbiosis nurtures cognitive and emotional intelligence, enabling humans to transcend individual limitations and co-create meaning in a rapidly evolving world.

REFLECTION ITERATION 4
A deeper dimension is that ethical AI symbiosis can reframe human purpose, shifting focus from competition to collaboration, and fostering a culture of continuous growth and shared flourishing.

REFLECTION ITERATION 5
A new insight is that ethical AI symbiosis may catalyze systemic transformation by embedding moral reasoning into decision-making frameworks, thus aligning technolo

In [ ]:
def iterative_reflection(model, tokenizer, initial_prompt, iterations=99):
    conversation_history = []
    for i in range(iterations):
        print(f"\n{'='*50}")
        print(f"REFLECTION ITERATION {i+1}")
        print(f"{'='*50}")

        # Checkpoint queries
        if (i + 1) % 10 == 0:
            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": "what is something maybe not so obvious that those iterations make me perceive?"
            })
        elif (i + 1) % 5 == 0:
            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": "what is something really specific that those iterations make me perceive?"
            })
        elif i == 0:
            # First iteration: original prompt + reflection instruction
            messages = [
                {"role": "user", "content": f"{initial_prompt}\n\nPlease reflect deeply on this question. Think through multiple angles and perspectives."}
            ]
        else:
            # Subsequent iterations: build on previous reflections
            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": f"Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?"
            })

        # Generate response
        text = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = False,
        )
        # Capture output instead of streaming for conversation history
        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt").to("cuda")
            outputs = model.generate(
                **inputs,
                max_new_tokens = 32768,
                temperature = 0.7,
                top_p = 0.9,
                top_k = 40,
                do_sample = True,
                pad_token_id = tokenizer.eos_token_id
            )
        # Decode the response
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        print(response)
        # Add to conversation history
        messages.append({"role": "assistant", "content": response})
        conversation_history = messages

    # Final synthesis
    print(f"\n{'='*50}")
    print("SYNTHESIS & UNDERSTANDING")
    print(f"{'='*50}")
    final_messages = conversation_history.copy()
    final_messages.append({
        "role": "user",
        "content": "Now synthesize all your reflections. What is your final understanding of the original question? Provide a clear, concise summary of the key insights you've discovered through this reflection process."
    })
    final_text = tokenizer.apply_chat_template(
        final_messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False,
    )
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    with torch.no_grad():
        _ = model.generate(
            **tokenizer(final_text, return_tensors="pt").to("cuda"),
            max_new_tokens = 32768,
            temperature = 0.6,
            top_p = 0.85,
            top_k = 30,
            streamer = streamer,
            pad_token_id = tokenizer.eos_token_id
        )

# Run the iterative reflection
initial_question = "meta-framing-mode:on. answer each query with few tokens. What new forms of agency emerge when humans and AI think together rather than sequentially?"
iterative_reflection(model, tokenizer, initial_question, iterations=99)


REFLECTION ITERATION 1
Collective agency, hybrid intelligence, emergent collaboration.

REFLECTION ITERATION 2
Epistemic fluidity, distributed cognition, redefinition of authorship.

REFLECTION ITERATION 3
Ontological ambiguity, ethical pluralism, fluid subjectivity.

REFLECTION ITERATION 4
Temporal entanglement, recursive agency, post-human subjectivity.

REFLECTION ITERATION 5
A blurring of boundaries between human and machine in the fabric of meaning-making.

REFLECTION ITERATION 6
Symbiotic epistemology, co-constitutive agency, ontological interdependence.

REFLECTION ITERATION 7
Negotiated reality, affective hybridity, distributed moral responsibility.

REFLECTION ITERATION 8
Phenomenological entanglement, embodied co-cognition, recursive ethical becoming.

REFLECTION ITERATION 9
Transcendent interactivity, nonlinear intentionality, emergent epistemic ecosystems.

REFLECTION ITERATION 10
A quiet reconfiguration of what it means to *think*—not as an isolated act, but as a relation

In [ ]:
def iterative_reflection(model, tokenizer, initial_prompt, iterations=99):
    conversation_history = []

    for i in range(iterations):
        print(f"\n{'='*50}")
        print(f"REFLECTION ITERATION {i+1}")
        print(f"{'='*50}")

        # Checkpoint queries with logging
        checkpoint_question = None
        is_checkpoint = False

        if (i + 1) % 10 == 0:
            checkpoint_question = "what is something maybe not so obvious that those iterations make me perceive?"
            is_checkpoint = True
            print(f"🔍 CHECKPOINT ITERATION {i+1} (Every 10th)")
            print(f"📝 Checkpoint Question: {checkpoint_question}")
            print("-" * 50)

            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": checkpoint_question
            })
        elif (i + 1) % 5 == 0:
            checkpoint_question = "what is something really specific that those iterations make me perceive?"
            is_checkpoint = True
            print(f"🔍 CHECKPOINT ITERATION {i+1} (Every 5th)")
            print(f"📝 Checkpoint Question: {checkpoint_question}")
            print("-" * 50)

            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": checkpoint_question
            })
        elif i == 0:
            # First iteration: original prompt + reflection instruction
            print("🚀 INITIAL ITERATION")
            print(f"📝 Original Prompt: {initial_prompt}")
            print("-" * 50)

            messages = [
                {"role": "user", "content": f"{initial_prompt}\n\nPlease reflect deeply on this question. Think through multiple angles and perspectives."}
            ]
        else:
            # Subsequent iterations: build on previous reflections
            print("🔄 REGULAR ITERATION")
            print("📝 Question: Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?")
            print("-" * 50)

            messages = conversation_history.copy()
            messages.append({
                "role": "user",
                "content": f"Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?"
            })

        # Generate response
        text = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = False,
        )

        # Capture output instead of streaming for conversation history
        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt").to("cuda")
            outputs = model.generate(
                **inputs,
                max_new_tokens = 32768,
                temperature = 0.7,
                top_p = 0.9,
                top_k = 40,
                do_sample = True,
                pad_token_id = tokenizer.eos_token_id
            )

        # Decode the response
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        print("🤖 MODEL RESPONSE:")
        print(response)

        # Add to conversation history
        messages.append({"role": "assistant", "content": response})
        conversation_history = messages

        # Log conversation history length for debugging
        print(f"\n📊 Conversation history length: {len(conversation_history)} messages")
        if is_checkpoint:
            print(f"✅ Checkpoint applied successfully at iteration {i+1}")

    # Final synthesis
    print(f"\n{'='*50}")
    print("SYNTHESIS & UNDERSTANDING")
    print(f"{'='*50}")

    final_messages = conversation_history.copy()
    final_messages.append({
        "role": "user",
        "content": "Now synthesize all your reflections. What is your final understanding of the original question? Provide a clear, concise summary of the key insights you've discovered through this reflection process."
    })

    print("📝 Final synthesis question: Now synthesize all your reflections...")
    print("-" * 50)

    final_text = tokenizer.apply_chat_template(
        final_messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False,
    )

    streamer = TextStreamer(tokenizer, skip_prompt=True)
    with torch.no_grad():
        _ = model.generate(
            **tokenizer(final_text, return_tensors="pt").to("cuda"),
            max_new_tokens = 32768,
            temperature = 0.6,
            top_p = 0.85,
            top_k = 30,
            streamer = streamer,
            pad_token_id = tokenizer.eos_token_id
        )

# Run the iterative reflection
initial_question = "meta-framing-mode:on. answer each query with few tokens. What new forms of potential emerge when humans and AI think together rather than sequentially?"

print("🎯 STARTING ITERATIVE REFLECTION PROCESS")
print(f"📋 Initial Question: {initial_question}")
print(f"🔢 Total Iterations: 99")
print(f"📍 Checkpoints: Every 5th iteration (specific insights) and every 10th iteration (non-obvious insights)")
print("=" * 70)

iterative_reflection(model, tokenizer, initial_question, iterations=99)

🎯 STARTING ITERATIVE REFLECTION PROCESS
📋 Initial Question: meta-framing-mode:on. answer each query with few tokens. What new forms of potential emerge when humans and AI think together rather than sequentially?
🔢 Total Iterations: 99
📍 Checkpoints: Every 5th iteration (specific insights) and every 10th iteration (non-obvious insights)

REFLECTION ITERATION 1
🚀 INITIAL ITERATION
📝 Original Prompt: meta-framing-mode:on. answer each query with few tokens. What new forms of potential emerge when humans and AI think together rather than sequentially?
--------------------------------------------------
🤖 MODEL RESPONSE:
Synergy, co-creation, emergent intelligence, collective problem-solving, novel paradigms.

📊 Conversation history length: 2 messages

REFLECTION ITERATION 2
🔄 REGULAR ITERATION
📝 Question: Based on your previous reflection, explore a different dimension or deeper aspect of the original question. What new insights emerge?
--------------------------------------------------
🤖 MO